<a href="https://colab.research.google.com/github/renanmath/imersao-dados-desafio-final/blob/main/Alura_ImersaoDados_modelos_preditivos_(parte2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nos capítulos anteriores...

No primeiro arquivo fizemos uma análise exploratória dos dados dos experimentos e no segundo arquivo, analizamos os dados dos resultados dos experimentos. No terceiro arquivo criamos alguns modelos preditivos para resolver alguns problemas propostos. Nesse quarto arquivo, propomos e resolvemos novos problemas
Mas antes, precisamos carregar novamente os bancos de dados.

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore') #ignora as mensagens de aviso

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [3]:
url_dados = 'https://github.com/alura-cursos/imersaodados3/blob/main/dados/dados_experimentos.zip?raw=true'
df = pd.read_csv(url_dados, compression = 'zip')
df.head()

,id,tratamento,tempo,dose,droga,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,g-10,g-11,g-12,g-13,g-14,g-15,g-16,g-17,g-18,g-19,g-20,g-21,g-22,g-23,g-24,g-25,g-26,g-27,g-28,g-29,g-30,g-31,g-32,g-33,g-34,...,c-60,c-61,c-62,c-63,c-64,c-65,c-66,c-67,c-68,c-69,c-70,c-71,c-72,c-73,c-74,c-75,c-76,c-77,c-78,c-79,c-80,c-81,c-82,c-83,c-84,c-85,c-86,c-87,c-88,c-89,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,com_droga,24,D1,b68db1d53,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,-0.0326,0.5548,-0.0921,1.1830,0.1530,0.5574,-0.4015,0.1789,-0.6528,-0.7969,0.6342,0.1778,-0.3694,-0.5688,-1.1360,-1.1880,0.6940,0.4393,0.2664,0.1907,0.1628,-0.2853,0.5819,0.2934,-0.5584,-0.0916,-0.3010,-0.1537,...,0.4805,0.4965,0.3680,0.8427,0.1042,0.1403,0.1758,1.2570,-0.5979,1.2250,-0.0553,0.7351,0.5810,0.9590,0.2427,0.0495,0.4141,0.8432,0.6162,-0.7318,1.2120,0.6362,-0.4427,0.1288,1.4840,0.1799,0.5367,-0.1111,-1.0120,0.6685,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,com_droga,72,D1,df89a8e5a,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,0.3372,-0.4047,0.8507,-1.1520,-0.4201,-0.0958,0.4590,0.0803,0.2250,0.5293,0.2839,-0.3494,0.2883,0.9449,-0.1646,-0.2657,-0.3372,0.3135,-0.4316,0.4773,0.2075,-0.4216,-0.1161,-0.0499,-0.2627,0.9959,-0.2483,0.2655,...,0.4083,0.0319,0.3905,0.7099,0.2912,0.4151,-0.2840,-0.3104,-0.6373,0.2887,-0.0765,0.2539,0.4443,0.5932,0.2031,0.7639,0.5499,-0.3322,-0.0977,0.4329,-0.2782,0.7827,0.5934,0.3402,0.1499,0.4420,0.9366,0.8193,-0.4236,0.3192,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,com_droga,48,D1,18bb41b2c,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,0.2155,0.0065,1.2300,-0.4797,-0.5631,-0.0366,-1.8300,0.6057,-0.3278,0.6042,-0.3075,-0.1147,-0.0570,-0.0799,-0.8181,-1.5320,0.2307,0.4901,0.4780,-1.3970,4.6240,-0.0437,1.2870,-1.8530,0.6069,0.4290,0.1783,0.0018,...,-0.5477,-0.7576,-0.0444,0.1894,-0.0014,-2.3640,-0.4682,0.1210,-0.5177,-0.0604,0.1682,-0.4436,0.4963,0.1363,0.3335,0.9760,-0.0427,-0.1235,0.0959,0.0690,-0.9416,-0.7548,-0.1109,-0.6272,0.3019,0.1172,0.1093,-0.3113,0.3019,-0.0873,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,com_droga,48,D1,8c7f86626,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,0.1792,-0.1321,-1.0600,-0.8269,-0.3584,-0.8511,-0.5844,-2.5690,0.8183,-0.0532,-0.8554,0.1160,-2.3520,2.1200,-1.1580,-0.7191,-0.8004,-1.4670,-0.0107,-0.8995,0.2406,-0.2479,-1.0890,-0.7575,0.0881,-2.7370,0.8745,0.5787,...,-2.1220,-0.3752,-2.3820,-3.7350,-2.9740,-1.4930,-1.6600,-3.1660,0.2816,-0.2990,-1.1870,-0.5044,-1.7750,-1.6120,-0.9215,-1.0810,-3.0520,-3.4470,-2.7740,-1.8460,-0.5568,-3.3960,-2.9510,-1.1550,-3.2620,-1.5390,-2.4600,-0.9417,-1.5550,0.2431,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,com_droga,72,D2,7cbed3131,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,-0.1498,-0.8789,0.8630,-0.2219,-0.5121,-0.9577,1.1750,0.2042,0.1970,0.1244,-1.7090,-0.3543,-0.5160,-0.3330,-0.2685,0.7649,0.2057,1.3720,0.6835,0.8056,-0.3754,-1.2090,0.2965,-0.0712,0.6389,0.6674,-0.0783,1.1740,...,-0.2274,0.3215,0.1535,-0.4640,-0.5943,0.3973,0.1500,0.5178,0.5159,0.6091,0.1813,-0.4249,0.7832,0.6529,0.5648,0.4817,0.0587,0.5303,0.6376,-0.3966,-1.4950,-0.9625,-0.0541,0.6273,0.4563,0.0698,0.8134,0.1924,0.6054,-0.1824,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [4]:
#Renomeando as colunas, eliminando '-'
colunas = df.columns.to_numpy()  #cria um array com os nomes das colunas
for i in range(0,colunas.shape[0]):
   colunas[i] = colunas[i].replace('-','')  #faz um loop percorrendo os índices do array, trocando '-' por ''
df.columns = colunas  #renomeia as colunas
df.rename(columns= {'droga':'composto'}, inplace = True) #renomeia a coluna 'droga' para 'composto'
df.tail() #mostra os 5 últimos regristos

,id,tratamento,tempo,dose,composto,g0,g1,g2,g3,g4,g5,g6,g7,g8,g9,g10,g11,g12,g13,g14,g15,g16,g17,g18,g19,g20,g21,g22,g23,g24,g25,g26,g27,g28,g29,g30,g31,g32,g33,g34,...,c60,c61,c62,c63,c64,c65,c66,c67,c68,c69,c70,c71,c72,c73,c74,c75,c76,c77,c78,c79,c80,c81,c82,c83,c84,c85,c86,c87,c88,c89,c90,c91,c92,c93,c94,c95,c96,c97,c98,c99
23809,id_fffb1ceed,com_droga,24,D2,df1d0a5a1,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,0.5773,0.3055,-0.4726,0.1269,0.2531,0.1730,-0.4532,-1.0790,0.2474,-0.4550,0.3588,0.1600,-0.7362,-0.1103,0.8550,-0.4139,0.5541,0.2310,-0.5573,-0.4397,-0.9260,-0.2424,-0.6686,0.2326,0.6456,0.0136,-0.5141,-0.6320,0.7166,...,0.0807,0.4024,-0.0895,0.4420,0.9641,-0.1846,-0.0014,0.3154,-0.2071,-0.6158,-0.2977,0.0992,0.6838,0.5259,0.7882,0.3119,-0.7697,0.2203,-1.0710,0.5979,0.0848,-0.2555,0.6293,1.1660,0.3329,0.2754,0.4108,-0.1252,-0.2340,0.2267,0.1969,0.0262,-0.8121,0.3434,0.5372,-0.3246,0.0631,0.9171,0.5258,0.4680
23810,id_fffb70c0c,com_droga,24,D2,ecf3b6b74,-1.3260,0.3478,-0.3743,0.9905,-0.7178,0.6621,-0.2252,-0.5565,0.5112,0.6727,-0.1851,2.8650,-0.2140,-0.6153,0.8362,0.5584,-0.2589,0.1292,0.0148,0.0949,-0.2182,-0.9235,0.0749,-1.5910,-0.8359,-0.9217,0.3013,0.1716,0.0880,0.1842,0.1835,0.5436,-0.0533,-0.0491,0.9543,...,0.1410,1.2640,-0.8663,0.8129,-0.1514,-0.4652,-0.7390,-1.3270,0.9925,1.0570,-0.3355,-0.2555,0.8219,0.6995,-0.2942,0.2408,-0.7781,-0.0929,-0.0329,0.0781,-1.4440,-0.0921,0.3188,-1.1080,0.4895,-0.2144,1.0960,0.3335,0.4444,-1.1130,0.4286,0.4426,0.0423,-0.3195,-0.8086,-0.9798,-0.2084,-0.1224,-0.2715,0.3689
23811,id_fffc1c3f4,com_controle,48,D2,cacb2b860,0.3942,0.3756,0.3109,-0.7389,0.5505,-0.0159,-0.2541,0.1745,-0.0340,0.4865,-0.1854,0.0716,0.1729,-0.0434,0.1542,-0.2192,-0.0302,-0.4218,0.4057,-0.5372,0.1521,-0.2651,0.2310,-0.8101,0.4943,0.6905,-0.3720,-1.4110,0.4516,1.2300,-0.1949,-1.3280,-0.4276,-0.0040,-0.3086,...,0.6845,0.7127,0.7294,0.4718,-0.2020,0.2783,0.4934,0.4144,0.5449,1.4690,-0.6142,0.6068,0.3434,0.9880,-0.0468,-0.1882,-0.0087,-0.0356,0.5718,0.4971,0.1057,0.2111,0.6992,0.0708,0.6169,0.2248,0.5994,0.2689,0.0305,1.2320,0.5409,0.3755,0.7343,0.2807,0.4116,0.6422,0.2256,0.7592,0.6656,0.3808
23812,id_fffcb9e7c,com_droga,24,D1,8b87a7a83,0.6660,0.2324,0.4392,0.2044,0.8531,-0.0343,0.0323,0.0463,0.4299,-0.7985,0.5742,0.1421,2.2700,0.2046,0.5363,-1.7330,0.1450,0.6097,0.2024,0.9865,-0.7805,0.9608,0.3440,2.7650,0.4925,0.6698,0.2374,-0.3372,0.8771,-2.6560,-0.2000,-0.2043,0.6797,-0.0248,-0.0927,...,0.3360,-0.6136,0.5011,0.9261,0.4419,0.0295,0.4220,0.4677,-0.1184,0.4524,0.4135,0.1356,-0.5801,0.0411,1.0240,1.0340,-0.0270,-0.4194,0.7403,-0.6793,-0.1423,0.7307,0.7946,-0.0650,0.9038,0.2324,0.9676,1.0940,0.4290,0.5187,-0.1105,0.4258,-0.2012,0.1506,1.5230,0.7101,0.1732,0.7015,-0.6290,0.0740
23813,id_ffffdd77b,com_droga,72,D1,972f41291,-0.8598,1.0240,-0.1361,0.7952,-0.3611,-3.6750,-1.2420,0.9146,3.0790,1.2460,1.9460,1.4370,2.9780,2.2370,-0.6818,0.6870,-1.1060,0.0182,-0.9247,-0.0738,-0.1919,-0.7722,-1.4050,-1.0050,-1.1170,-0.5293,-1.1720,-0.2885,0.1599,-0.4250,0.3591,-0.1420,-0.9530,-0.2005,-1.8340,...,-6.9170,-1.8640,-2.5090,-4.7130,-1.7250,-3.8650,-3.0800,-4.1530,-1.2030,-1.1690,-4.1460,-1.2670,-1.1300,-2.4390,0.1591,-2.2490,-2.5860,-1.9520,-2.1810,-4.6690,-3.9450,-2.9560,-2.7930,-2.1560,-2.4100,-1.8190,-3.3480,-0.1414,-2.6430,-2.5810,-3.3890,-1.7450,-6.6300,-4.0950,-7.3860,-1.4160,-3.5770,-0.4775,-2.1500,-4.2520


In [5]:
df_resultados = pd.read_csv('https://github.com/alura-cursos/imersaodados3/blob/main/dados/dados_resultados.csv?raw=true')
df_resultados

,id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,...,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhibitor,retinoid_receptor_agonist,retinoid_receptor_antagonist,rho_associated_kinase_inhibitor,ribonucleoside_reductase_inhibitor,rna_polymerase_inhibitor,serotonin_receptor_agonist,serotonin_receptor_antagonist,serotonin_reuptake_inhibitor,sigma_receptor_agonist,sigma_receptor_antagonist,smoothened_receptor_antagonist,sodium_channel_inhibitor,sphingosine_receptor_agonist,src_inhibitor,steroid,syk_inhibitor,tachykinin_antagonist,tgf-beta_receptor_inhibitor,thrombin_inhibitor,thymidylate_synthase_inhibitor,tlr_agonist,tlr_antagonist,tnf_inhibitor,topoisomerase_inhibitor,transient_receptor_potential_channel_antagonist,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
23810,id_fffb70c0c,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
23811,id_fffc1c3f4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
23812,id_fffcb9e7c,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
df_resultados['n_moa'] = df_resultados.drop('id', axis=1).sum(axis=1)

In [7]:
df_resultados['ativo_moa'] = (df_resultados['n_moa'] != 0)*1
df_resultados.head()

,id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,...,radiopaque_medium,raf_inhibitor,ras_gtpase_inhibitor,retinoid_receptor_agonist,retinoid_receptor_antagonist,rho_associated_kinase_inhibitor,ribonucleoside_reductase_inhibitor,rna_polymerase_inhibitor,serotonin_receptor_agonist,serotonin_receptor_antagonist,serotonin_reuptake_inhibitor,sigma_receptor_agonist,sigma_receptor_antagonist,smoothened_receptor_antagonist,sodium_channel_inhibitor,sphingosine_receptor_agonist,src_inhibitor,steroid,syk_inhibitor,tachykinin_antagonist,tgf-beta_receptor_inhibitor,thrombin_inhibitor,thymidylate_synthase_inhibitor,tlr_agonist,tlr_antagonist,tnf_inhibitor,topoisomerase_inhibitor,transient_receptor_potential_channel_antagonist,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,n_moa,ativo_moa
0,id_000644bb2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
1,id_000779bfc,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1
3,id_0015fd391,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1


In [8]:
df_total = pd.merge(df, df_resultados[['id','n_moa', 'ativo_moa']], on='id')
df_total.head()

,id,tratamento,tempo,dose,composto,g0,g1,g2,g3,g4,g5,g6,g7,g8,g9,g10,g11,g12,g13,g14,g15,g16,g17,g18,g19,g20,g21,g22,g23,g24,g25,g26,g27,g28,g29,g30,g31,g32,g33,g34,...,c62,c63,c64,c65,c66,c67,c68,c69,c70,c71,c72,c73,c74,c75,c76,c77,c78,c79,c80,c81,c82,c83,c84,c85,c86,c87,c88,c89,c90,c91,c92,c93,c94,c95,c96,c97,c98,c99,n_moa,ativo_moa
0,id_000644bb2,com_droga,24,D1,b68db1d53,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,-0.0326,0.5548,-0.0921,1.1830,0.1530,0.5574,-0.4015,0.1789,-0.6528,-0.7969,0.6342,0.1778,-0.3694,-0.5688,-1.1360,-1.1880,0.6940,0.4393,0.2664,0.1907,0.1628,-0.2853,0.5819,0.2934,-0.5584,-0.0916,-0.3010,-0.1537,...,0.3680,0.8427,0.1042,0.1403,0.1758,1.2570,-0.5979,1.2250,-0.0553,0.7351,0.5810,0.9590,0.2427,0.0495,0.4141,0.8432,0.6162,-0.7318,1.2120,0.6362,-0.4427,0.1288,1.4840,0.1799,0.5367,-0.1111,-1.0120,0.6685,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176,1,1
1,id_000779bfc,com_droga,72,D1,df89a8e5a,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,0.3372,-0.4047,0.8507,-1.1520,-0.4201,-0.0958,0.4590,0.0803,0.2250,0.5293,0.2839,-0.3494,0.2883,0.9449,-0.1646,-0.2657,-0.3372,0.3135,-0.4316,0.4773,0.2075,-0.4216,-0.1161,-0.0499,-0.2627,0.9959,-0.2483,0.2655,...,0.3905,0.7099,0.2912,0.4151,-0.2840,-0.3104,-0.6373,0.2887,-0.0765,0.2539,0.4443,0.5932,0.2031,0.7639,0.5499,-0.3322,-0.0977,0.4329,-0.2782,0.7827,0.5934,0.3402,0.1499,0.4420,0.9366,0.8193,-0.4236,0.3192,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371,0,0
2,id_000a6266a,com_droga,48,D1,18bb41b2c,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,0.2155,0.0065,1.2300,-0.4797,-0.5631,-0.0366,-1.8300,0.6057,-0.3278,0.6042,-0.3075,-0.1147,-0.0570,-0.0799,-0.8181,-1.5320,0.2307,0.4901,0.4780,-1.3970,4.6240,-0.0437,1.2870,-1.8530,0.6069,0.4290,0.1783,0.0018,...,-0.0444,0.1894,-0.0014,-2.3640,-0.4682,0.1210,-0.5177,-0.0604,0.1682,-0.4436,0.4963,0.1363,0.3335,0.9760,-0.0427,-0.1235,0.0959,0.0690,-0.9416,-0.7548,-0.1109,-0.6272,0.3019,0.1172,0.1093,-0.3113,0.3019,-0.0873,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931,3,1
3,id_0015fd391,com_droga,48,D1,8c7f86626,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,0.1792,-0.1321,-1.0600,-0.8269,-0.3584,-0.8511,-0.5844,-2.5690,0.8183,-0.0532,-0.8554,0.1160,-2.3520,2.1200,-1.1580,-0.7191,-0.8004,-1.4670,-0.0107,-0.8995,0.2406,-0.2479,-1.0890,-0.7575,0.0881,-2.7370,0.8745,0.5787,...,-2.3820,-3.7350,-2.9740,-1.4930,-1.6600,-3.1660,0.2816,-0.2990,-1.1870,-0.5044,-1.7750,-1.6120,-0.9215,-1.0810,-3.0520,-3.4470,-2.7740,-1.8460,-0.5568,-3.3960,-2.9510,-1.1550,-3.2620,-1.5390,-2.4600,-0.9417,-1.5550,0.2431,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154,0,0
4,id_001626bd3,com_droga,72,D2,7cbed3131,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,-0.1498,-0.8789,0.8630,-0.2219,-0.5121,-0.9577,1.1750,0.2042,0.1970,0.1244,-1.7090,-0.3543,-0.5160,-0.3330,-0.2685,0.7649,0.2057,1.3720,0.6835,0.8056,-0.3754,-1.2090,0.2965,-0.0712,0.6389,0.6674,-0.0783,1.1740,...,0.1535,-0.4640,-0.5943,0.3973,0.1500,0.5178,0.5159,0.6091,0.1813,-0.4249,0.7832,0.6529,0.5648,0.4817,0.0587,0.5303,0.6376,-0.3966,-1.4950,-0.9625,-0.0541,0.6273,0.4563,0.0698,0.8134,0.1924,0.6054,-0.1824,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125,1,1


In [9]:
df_total['Eh controle'] = (df_total['tratamento'] == 'com_controle')*1
df_total['Tempo eh 24'] = (df_total['tempo'] == 24)*1
df_total['Tempo eh 48'] = (df_total['tempo'] == 48)*1
df_total['Tempo eh 72'] = (df_total['tempo'] == 72)*1

#Máquinas preditivas

## Problema 5: Prevendo se o composto ativa dois MOAs específicos

No Problema 3 criamos modelos que previam se um composto ativava um MOA específico. Aqui faremos algo semelhante, mas agora com dois MOAs. 
Como no problema 3, queremos uma alta precisão para positivos e negativos. 


Vejamos novamente a lista dos MOAs mais ativados fazendo a soma das colunas em toda a tabela df_resultados.

In [10]:
df_resultados.iloc[:,1:].sum(axis = 0).sort_values(ascending = False)

n_moa                                         16844
ativo_moa                                     14447
nfkb_inhibitor                                  832
proteasome_inhibitor                            726
cyclooxygenase_inhibitor                        435
                                              ...  
steroid                                           6
nicotinic_receptor_agonist                        6
elastase_inhibitor                                6
erbb2_inhibitor                                   1
atp-sensitive_potassium_channel_antagonist        1
Length: 208, dtype: int64

Assim, os MOAs mais ativados são nfkb_inhibitor e proteasome_inhibitor. Usaremos esses dois em nosso modelo.
Precisamos criar um novo data frame usando merge, para incluir as informações sobre expressões gênicas e uma coluna que nos informará se os dois MOAs foram ativados.

In [11]:
df_moa_ativado = pd.merge(df.iloc[:,0:200], df_resultados[['id','proteasome_inhibitor', 'nfkb_inhibitor']], on='id')
df_moa_ativado['ativou os dois'] = (df_moa_ativado['proteasome_inhibitor']+df_moa_ativado['nfkb_inhibitor'] ==2)*1
df_moa_ativado

,id,tratamento,tempo,dose,composto,g0,g1,g2,g3,g4,g5,g6,g7,g8,g9,g10,g11,g12,g13,g14,g15,g16,g17,g18,g19,g20,g21,g22,g23,g24,g25,g26,g27,g28,g29,g30,g31,g32,g33,g34,...,g158,g159,g160,g161,g162,g163,g164,g165,g166,g167,g168,g169,g170,g171,g172,g173,g174,g175,g176,g177,g178,g179,g180,g181,g182,g183,g184,g185,g186,g187,g188,g189,g190,g191,g192,g193,g194,proteasome_inhibitor,nfkb_inhibitor,ativou os dois
0,id_000644bb2,com_droga,24,D1,b68db1d53,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,-0.0326,0.5548,-0.0921,1.1830,0.1530,0.5574,-0.4015,0.1789,-0.6528,-0.7969,0.6342,0.1778,-0.3694,-0.5688,-1.1360,-1.1880,0.6940,0.4393,0.2664,0.1907,0.1628,-0.2853,0.5819,0.2934,-0.5584,-0.0916,-0.3010,-0.1537,...,-0.7799,-0.8062,-0.3239,0.2411,-1.0170,-0.5592,-0.4386,0.0590,0.3816,0.0817,-0.8457,0.5989,-0.4194,0.0858,0.7449,1.6820,-0.2471,-1.4130,-0.2031,-0.2256,-0.6217,0.4969,0.1109,-0.3284,0.2469,-0.2786,0.2318,-0.5274,0.5698,-0.2100,1.4050,-0.1078,0.1064,-0.2164,0.0917,0.4796,-0.1509,0,0,0
1,id_000779bfc,com_droga,72,D1,df89a8e5a,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,0.3372,-0.4047,0.8507,-1.1520,-0.4201,-0.0958,0.4590,0.0803,0.2250,0.5293,0.2839,-0.3494,0.2883,0.9449,-0.1646,-0.2657,-0.3372,0.3135,-0.4316,0.4773,0.2075,-0.4216,-0.1161,-0.0499,-0.2627,0.9959,-0.2483,0.2655,...,0.1049,-0.1423,0.0882,1.1880,-0.5056,-0.0163,-1.4020,0.6522,1.2160,-0.0478,0.3713,0.2215,0.2610,0.3841,-0.1476,-1.2850,-0.6565,0.2697,0.0919,0.2341,1.3360,-0.1854,0.0325,0.1416,-0.2420,-0.2166,-0.5338,0.1229,-0.2280,-0.3149,0.7871,-0.1145,-0.2067,-0.3810,0.3166,-1.4640,-0.1521,0,0,0
2,id_000a6266a,com_droga,48,D1,18bb41b2c,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,0.2155,0.0065,1.2300,-0.4797,-0.5631,-0.0366,-1.8300,0.6057,-0.3278,0.6042,-0.3075,-0.1147,-0.0570,-0.0799,-0.8181,-1.5320,0.2307,0.4901,0.4780,-1.3970,4.6240,-0.0437,1.2870,-1.8530,0.6069,0.4290,0.1783,0.0018,...,0.3011,0.4400,1.7920,-4.1810,2.1840,-0.9974,0.2307,-0.8124,0.8455,-2.7010,-0.9168,-1.1480,0.0825,-0.6908,-0.4245,-0.3494,-0.1320,1.1870,-2.6240,-1.6640,0.1667,-1.2800,0.7258,-0.3007,0.5677,0.4284,-0.2485,-2.1940,-0.0611,0.0544,3.6780,-0.9055,2.0350,1.1970,-0.3382,0.4036,1.1240,0,0,0
3,id_0015fd391,com_droga,48,D1,8c7f86626,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,0.1792,-0.1321,-1.0600,-0.8269,-0.3584,-0.8511,-0.5844,-2.5690,0.8183,-0.0532,-0.8554,0.1160,-2.3520,2.1200,-1.1580,-0.7191,-0.8004,-1.4670,-0.0107,-0.8995,0.2406,-0.2479,-1.0890,-0.7575,0.0881,-2.7370,0.8745,0.5787,...,4.2110,-0.5516,0.8118,-0.8374,1.0510,1.1430,0.9099,-0.2411,-1.3680,1.0670,0.2782,1.4610,-0.5527,-1.0070,-0.0910,-1.0610,0.7465,3.4580,-0.5228,0.3133,-3.1620,-1.2130,0.0650,1.6530,-0.0022,0.0864,0.8989,-0.8746,-1.4210,-0.2400,-0.1266,-2.5660,3.5620,2.5130,-0.1470,0.3183,-0.0179,0,0,0
4,id_001626bd3,com_droga,72,D2,7cbed3131,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,-0.1498,-0.8789,0.8630,-0.2219,-0.5121,-0.9577,1.1750,0.2042,0.1970,0.1244,-1.7090,-0.3543,-0.5160,-0.3330,-0.2685,0.7649,0.2057,1.3720,0.6835,0.8056,-0.3754,-1.2090,0.2965,-0.0712,0.6389,0.6674,-0.0783,1.1740,...,0.9283,1.4940,-0.8757,-0.3667,0.4272,-0.9715,0.3501,0.3622,-0.8955,0.1902,1.1520,0.6745,0.6135,-1.4480,-0.6243,-1.6000,-0.0913,0.5282,1.6050,0.5905,-0.8969,-0.9344,-0.9038,-0.5732,-0.6177,0.1723,-0.6239,-0.8446,-0.8357,0.6745,0.5846,-0.0990,0.8706,0.4460,-0.4033,-0.5549,0.2064,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,com_droga,24,D2,df1d0a5a1,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,0.5773,0.3055,-0.4726,0.1269,0.2531,0.1730,-0.4532,-1.0790,0.2474,-0.4550,0.3588,0.1600,-0.7362,-0.1103,0.8550,-0.4139,0.5541,0.2310,-0.5573,-0.4397,-0.9260,-0.2424,-0.6686,0.2326,0.6456,0.0136,-0.5141,-0.6320,0.7

In [12]:
df_moa_ativado['Eh controle'] = (df_moa_ativado['tratamento'] == 'com_controle')*1
df_moa_ativado['Tempo eh 24'] = (df_moa_ativado['tempo'] == 24)*1
df_moa_ativado['Tempo eh 48'] = (df_moa_ativado['tempo'] == 48)*1
df_moa_ativado['Tempo eh 72'] = (df_moa_ativado['tempo'] == 72)*1

Criando as variáveis:

In [13]:
y = df_moa_ativado['ativou os dois']
x_aux = df_moa_ativado.drop(['id', 'tratamento','composto' ,'dose', 'tempo', 'proteasome_inhibitor', 'nfkb_inhibitor','ativou os dois'], axis = 1)
x = x_aux.drop(x_aux.columns[200:873])

In [14]:
x

,g0,g1,g2,g3,g4,g5,g6,g7,g8,g9,g10,g11,g12,g13,g14,g15,g16,g17,g18,g19,g20,g21,g22,g23,g24,g25,g26,g27,g28,g29,g30,g31,g32,g33,g34,g35,g36,g37,g38,g39,...,g159,g160,g161,g162,g163,g164,g165,g166,g167,g168,g169,g170,g171,g172,g173,g174,g175,g176,g177,g178,g179,g180,g181,g182,g183,g184,g185,g186,g187,g188,g189,g190,g191,g192,g193,g194,Eh controle,Tempo eh 24,Tempo eh 48,Tempo eh 72
0,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,-0.0326,0.5548,-0.0921,1.1830,0.1530,0.5574,-0.4015,0.1789,-0.6528,-0.7969,0.6342,0.1778,-0.3694,-0.5688,-1.1360,-1.1880,0.6940,0.4393,0.2664,0.1907,0.1628,-0.2853,0.5819,0.2934,-0.5584,-0.0916,-0.3010,-0.1537,0.2198,0.2965,-0.5055,-0.5119,-0.2162,...,-0.8062,-0.3239,0.2411,-1.0170,-0.5592,-0.4386,0.0590,0.3816,0.0817,-0.8457,0.5989,-0.4194,0.0858,0.7449,1.6820,-0.2471,-1.4130,-0.2031,-0.2256,-0.6217,0.4969,0.1109,-0.3284,0.2469,-0.2786,0.2318,-0.5274,0.5698,-0.2100,1.4050,-0.1078,0.1064,-0.2164,0.0917,0.4796,-0.1509,0,1,0,0
1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,0.3372,-0.4047,0.8507,-1.1520,-0.4201,-0.0958,0.4590,0.0803,0.2250,0.5293,0.2839,-0.3494,0.2883,0.9449,-0.1646,-0.2657,-0.3372,0.3135,-0.4316,0.4773,0.2075,-0.4216,-0.1161,-0.0499,-0.2627,0.9959,-0.2483,0.2655,-0.2102,0.1656,0.5300,-0.2568,-0.0455,...,-0.1423,0.0882,1.1880,-0.5056,-0.0163,-1.4020,0.6522,1.2160,-0.0478,0.3713,0.2215,0.2610,0.3841,-0.1476,-1.2850,-0.6565,0.2697,0.0919,0.2341,1.3360,-0.1854,0.0325,0.1416,-0.2420,-0.2166,-0.5338,0.1229,-0.2280,-0.3149,0.7871,-0.1145,-0.2067,-0.3810,0.3166,-1.4640,-0.1521,0,0,0,1
2,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,0.2155,0.0065,1.2300,-0.4797,-0.5631,-0.0366,-1.8300,0.6057,-0.3278,0.6042,-0.3075,-0.1147,-0.0570,-0.0799,-0.8181,-1.5320,0.2307,0.4901,0.4780,-1.3970,4.6240,-0.0437,1.2870,-1.8530,0.6069,0.4290,0.1783,0.0018,-1.1800,0.1256,-0.1219,5.4470,1.0310,...,0.4400,1.7920,-4.1810,2.1840,-0.9974,0.2307,-0.8124,0.8455,-2.7010,-0.9168,-1.1480,0.0825,-0.6908,-0.4245,-0.3494,-0.1320,1.1870,-2.6240,-1.6640,0.1667,-1.2800,0.7258,-0.3007,0.5677,0.4284,-0.2485,-2.1940,-0.0611,0.0544,3.6780,-0.9055,2.0350,1.1970,-0.3382,0.4036,1.1240,0,0,1,0
3,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,0.1792,-0.1321,-1.0600,-0.8269,-0.3584,-0.8511,-0.5844,-2.5690,0.8183,-0.0532,-0.8554,0.1160,-2.3520,2.1200,-1.1580,-0.7191,-0.8004,-1.4670,-0.0107,-0.8995,0.2406,-0.2479,-1.0890,-0.7575,0.0881,-2.7370,0.8745,0.5787,-1.6740,-1.6720,-1.2690,3.0900,-0.3814,...,-0.5516,0.8118,-0.8374,1.0510,1.1430,0.9099,-0.2411,-1.3680,1.0670,0.2782,1.4610,-0.5527,-1.0070,-0.0910,-1.0610,0.7465,3.4580,-0.5228,0.3133,-3.1620,-1.2130,0.0650,1.6530,-0.0022,0.0864,0.8989,-0.8746,-1.4210,-0.2400,-0.1266,-2.5660,3.5620,2.5130,-0.1470,0.3183,-0.0179,0,0,1,0
4,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,-0.1498,-0.8789,0.8630,-0.2219,-0.5121,-0.9577,1.1750,0.2042,0.1970,0.1244,-1.7090,-0.3543,-0.5160,-0.3330,-0.2685,0.7649,0.2057,1.3720,0.6835,0.8056,-0.3754,-1.2090,0.2965,-0.0712,0.6389,0.6674,-0.0783,1.1740,-0.7110,-1.4470,1.0620,0.7888,-0.0848,...,1.4940,-0.8757,-0.3667,0.4272,-0.9715,0.3501,0.3622,-0.8955,0.1902,1.1520,0.6745,0.6135,-1.4480,-0.6243,-1.6000,-0.0913,0.5282,1.6050,0.5905,-0.8969,-0.9344,-0.9038,-0.5732,-0.6177,0.1723,-0.6239,-0.8446,-0.8357,0.6745,0.5846,-0.0990,0.8706,0.4460,-0.4033,-0.5549,0.2064,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,0.5773,0.3055,-0.4726,0.1269,0.2531,0.1730,-0.4532,-1.0790,0.2474,-0.4550,0.3588,0.1600,-0.7362,-0.1103,0.8550,-0.4139,0.5541,0.2310,-0.5573,-0.4397,-0.9260,-0.2424,-0.6686,0.2326,0.6456,0.0136,-0.5141,-0.6320,0.7166,-0.1736,0.3686,-0.1565,-0.7362,0.1318,...,0.4381,-1.1670,0.2147,-0.0811,-0.5493,-0.3679,

Separando os conjuntos de treino e conjunto de teste.

In [15]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x,y, test_size = 0.3, stratify = y)

Estamos prontos para começar a testar os modelos preditivos.
Como antes, vamos criar também um dataframe com informações sumarizadas das performaces de cada modelo. Assim:

In [16]:
lista_modelo = []
lista_score = []
lista_precision0 = [] 
lista_precision1 = []
lista_recall0 = []
lista_recall1 = []

### Regressão Logistica

In [17]:
modelo = LogisticRegression()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  99.90202939118265 %


In [18]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[6925    5]
 [   2  213]]


In [19]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6930
           1       0.98      0.99      0.98       215

    accuracy                           1.00      7145
   macro avg       0.99      0.99      0.99      7145
weighted avg       1.00      1.00      1.00      7145



### DecisionTreeClassifier 

In [20]:
modelo = DecisionTreeClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  99.52414275717285 %


In [21]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[6917   13]
 [  21  194]]


In [22]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6930
           1       0.94      0.90      0.92       215

    accuracy                           1.00      7145
   macro avg       0.97      0.95      0.96      7145
weighted avg       1.00      1.00      1.00      7145



### RandomFlorestClassifier

In [23]:
modelo = RandomForestClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  99.88803358992303 %


In [24]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[6922    8]
 [   0  215]]


In [25]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6930
           1       0.96      1.00      0.98       215

    accuracy                           1.00      7145
   macro avg       0.98      1.00      0.99      7145
weighted avg       1.00      1.00      1.00      7145



### LGBMClassifier

In [26]:
modelo = LGBMClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  99.8740377886634 %


In [27]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[6924    6]
 [   3  212]]


In [28]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6930
           1       0.97      0.99      0.98       215

    accuracy                           1.00      7145
   macro avg       0.99      0.99      0.99      7145
weighted avg       1.00      1.00      1.00      7145



### XGBClassifier

In [29]:
modelo = XGBClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  99.88803358992303 %


In [30]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[6923    7]
 [   1  214]]


In [31]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6930
           1       0.97      1.00      0.98       215

    accuracy                           1.00      7145
   macro avg       0.98      1.00      0.99      7145
weighted avg       1.00      1.00      1.00      7145



### GradientBoostingClassifier


In [32]:
modelo = GradientBoostingClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  99.84604618614415 %


In [33]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[6924    6]
 [   5  210]]


In [34]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6930
           1       0.97      0.98      0.97       215

    accuracy                           1.00      7145
   macro avg       0.99      0.99      0.99      7145
weighted avg       1.00      1.00      1.00      7145



### KNeighborsClassifier

In [35]:
modelo = KNeighborsClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  99.73407977606718 %


In [36]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[6911   19]
 [   0  215]]


In [37]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6930
           1       0.92      1.00      0.96       215

    accuracy                           1.00      7145
   macro avg       0.96      1.00      0.98      7145
weighted avg       1.00      1.00      1.00      7145



### Support Vector Machine

In [38]:
modelo = SVC()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  99.73407977606718 %


In [39]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[6924    6]
 [  13  202]]


In [40]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6930
           1       0.97      0.94      0.96       215

    accuracy                           1.00      7145
   macro avg       0.98      0.97      0.98      7145
weighted avg       1.00      1.00      1.00      7145



### Resumo

In [41]:
Tabela_Performace_Modelos = pd.DataFrame()
Tabela_Performace_Modelos['Modelo'] = lista_modelo
Tabela_Performace_Modelos['Score'] = lista_score
Tabela_Performace_Modelos['precision-0'] = lista_precision0
Tabela_Performace_Modelos['precision-1'] = lista_precision1
Tabela_Performace_Modelos['recall-0'] = lista_recall0
Tabela_Performace_Modelos['recall-1'] = lista_recall1

Tabela_Comparação_Modelos_Problema5 = Tabela_Performace_Modelos
Tabela_Comparação_Modelos_Problema5

,Modelo,Score,precision-0,precision-1,recall-0,recall-1
0,LogisticRegression,0.999020,0.999711,0.977064,0.999278,0.990698
1,DecisionTreeClassifier,0.995241,0.996973,0.937198,0.998124,0.902326
2,RandomForestClassifier,0.998880,1.000000,0.964126,0.998846,1.000000
3,LGBMClassifier,0.998740,0.999567,0.972477,0.999134,0.986047
4,XGBClassifier,0.998880,0.999856,0.968326,0.998990,0.995349
5,GradientBoostingClassifier,0.998460,0.999278,0.972222,0.999134,0.976744
6,KNeighborsClassifier,0.997341,1.000000,0.918803,0.997258,1.000000
7,SVC,0.997341,0.998126,0.971154,0.999134,0.939535


Percebemos então que todos os modelos tiveram excelente desempenho, considerando todas as métricas. 

## Problema 6: prevendo a expressão gênica g0 a partir dos MOAs ativados

Estamos agora interessante em uma nova classe de problemas: os problemas de regressão. Nesse problema 6, gostaríamos de tentar prever o valor da expressão gênica g0 a partir dos MOAs que foram ativados. 
Começamos importando novas bibliotecas:

In [42]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import  MLPRegressor

Criando nosso data frame, usando o merge:

In [43]:
df_g0 = pd.merge(df.drop(df.columns[6:], axis = 1), df_resultados, on='id')

In [44]:
df_g0

,id,tratamento,tempo,dose,composto,g0,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,...,radiopaque_medium,raf_inhibitor,ras_gtpase_inhibitor,retinoid_receptor_agonist,retinoid_receptor_antagonist,rho_associated_kinase_inhibitor,ribonucleoside_reductase_inhibitor,rna_polymerase_inhibitor,serotonin_receptor_agonist,serotonin_receptor_antagonist,serotonin_reuptake_inhibitor,sigma_receptor_agonist,sigma_receptor_antagonist,smoothened_receptor_antagonist,sodium_channel_inhibitor,sphingosine_receptor_agonist,src_inhibitor,steroid,syk_inhibitor,tachykinin_antagonist,tgf-beta_receptor_inhibitor,thrombin_inhibitor,thymidylate_synthase_inhibitor,tlr_agonist,tlr_antagonist,tnf_inhibitor,topoisomerase_inhibitor,transient_receptor_potential_channel_antagonist,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,n_moa,ativo_moa
0,id_000644bb2,com_droga,24,D1,b68db1d53,1.0620,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
1,id_000779bfc,com_droga,72,D1,df89a8e5a,0.0743,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,com_droga,48,D1,18bb41b2c,0.6280,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1
3,id_0015fd391,com_droga,48,D1,8c7f86626,-0.5138,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,com_droga,72,D2,7cbed3131,-0.3254,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,com_droga,24,D2,df1d0a5a1,0.1394,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
23810,id_fffb70c0c,com_droga,24,D2,ecf3b6b74,-1.3260,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
23811,id_fffc1c3f4,com_controle,48,D2,cacb2b860,0.3942,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
23812,id_fffcb9e7c,com_droga,24,D1,8b87a7a83,0.6660,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1


Criando nossas variáveis:

In [45]:
y = df_g0['g0']
x = df_g0.drop(['id', 'tratamento', 'tempo', 'dose', 'composto', 'g0'], axis = 1)

In [46]:
x

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,...,radiopaque_medium,raf_inhibitor,ras_gtpase_inhibitor,retinoid_receptor_agonist,retinoid_receptor_antagonist,rho_associated_kinase_inhibitor,ribonucleoside_reductase_inhibitor,rna_polymerase_inhibitor,serotonin_receptor_agonist,serotonin_receptor_antagonist,serotonin_reuptake_inhibitor,sigma_receptor_agonist,sigma_receptor_antagonist,smoothened_receptor_antagonist,sodium_channel_inhibitor,sphingosine_receptor_agonist,src_inhibitor,steroid,syk_inhibitor,tachykinin_antagonist,tgf-beta_receptor_inhibitor,thrombin_inhibitor,thymidylate_synthase_inhibitor,tlr_agonist,tlr_antagonist,tnf_inhibitor,topoisomerase_inhibitor,transient_receptor_potential_channel_antagonist,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,n_moa,ativo_moa
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
23810,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
23811,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
23812,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1


In [47]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x,y, test_size = 0.3)

In [48]:
lista_modelo = []
lista_score = []

### Regressão Linear

In [49]:
modelo = LinearRegression()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  -1.5191127387715377e+22 %


### DecisionTreeRegressor

In [50]:
modelo = DecisionTreeRegressor()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  42.82408507876403 %


### RandomFlorestRegressor

In [51]:
modelo = RandomForestRegressor()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  42.91012147375143 %


### XGBR Regressor

In [52]:
modelo = XGBRegressor()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

[19:25:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Score =  42.119040708065384 %


### LGBM Regressor

In [53]:
modelo = LGBMRegressor()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  41.5503912494968 %


### Gradiant Boosting Regressor

In [54]:
modelo = GradientBoostingRegressor()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  42.19580867600785 %


### KNeighbors Regressor

In [55]:
modelo = KNeighborsRegressor()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  34.736173148670304 %


### MLP Regressor

In [56]:
modelo = MLPRegressor()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  42.804499741678114 %


### Resumo

In [57]:
Tabela_Performace_Modelos = pd.DataFrame()
Tabela_Performace_Modelos['Modelo'] = lista_modelo
Tabela_Performace_Modelos['Score'] = lista_score

Tabela_Comparação_Modelos_Problema6 = Tabela_Performace_Modelos
Tabela_Comparação_Modelos_Problema6

,Modelo,Score
0,LinearRegression,-1.519113e+20
1,DecisionTreeRegressor,4.282409e-01
2,RandomForestRegressor,4.291012e-01
3,XGBRegressor,4.211904e-01
4,LGBMRegressor,4.155039e-01
5,GradientBoostingRegressor,4.219581e-01
6,KNeighborsRegressor,3.473617e-01
7,MLPRegressor,4.280450e-01


Concluímos então que para esse problema, os modelos tiveram um desempenho pífio. 
Isso talvez se explique pelo fato de que usamos variáveis categóricas (de fato, binárias) para tentar prever uma variável contínua. 

## Problema 7: prevendo a viabilidade celular a partir das expressões gênicas.

Queremos agora saber se é possível recupear as viabilidades celulares, sabendo as expressões gênicas. Como as variáveis c0-c99 são muito correlacionadas, basta verificar para uma delas, digamos, c0.

Criando as variáveis:

In [58]:
y = df['c0']
x_aux = df.drop(['id', 'tratamento', 'tempo', 'dose', 'composto'],  axis = 1)
x = x_aux.iloc[:,0:200]

In [59]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x,y, test_size = 0.3)

In [60]:
lista_modelo = []
lista_score = []

### Regressão Linear

In [61]:
modelo = LinearRegression()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  80.75826966907434 %


### DecisionTreeRegressor

In [62]:
modelo = DecisionTreeRegressor()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')
  


Score =  63.677903638685194 %


### RandomFlorestRegressor

In [63]:
modelo = RandomForestRegressor()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  82.41144193682692 %


### XGBR Regressor

In [64]:
modelo = XGBRegressor()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

[19:34:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Score =  82.84268528095438 %


### LGBM Regressor

In [65]:
modelo = LGBMRegressor()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  83.01265384799555 %


### Gradiant Boosting Regressor

In [66]:
modelo = GradientBoostingRegressor()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  82.82455517762588 %


### KNeighbors Regressor

In [67]:
modelo = KNeighborsRegressor()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  80.14757112986595 %


### MLP Regressor

In [68]:
modelo = MLPRegressor()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  75.14327537803901 %


### Resumo

In [69]:
Tabela_Performace_Modelos = pd.DataFrame()
Tabela_Performace_Modelos['Modelo'] = lista_modelo
Tabela_Performace_Modelos['Score'] = lista_score

Tabela_Comparação_Modelos_Problema7 = Tabela_Performace_Modelos
Tabela_Comparação_Modelos_Problema7

,Modelo,Score
0,LinearRegression,0.807583
1,DecisionTreeRegressor,0.636779
2,RandomForestRegressor,0.824114
3,XGBRegressor,0.828427
4,LGBMRegressor,0.830127
5,GradientBoostingRegressor,0.828246
6,KNeighborsRegressor,0.801476
7,MLPRegressor,0.751433


Vemos então que, com excessão dos modelos de árvore de decisão e MLP, todos os modelos apresentaram desempenho satisfatório. 

## Problema 8: prevendo o composto a partir das expressões gênicas

Deixamos agora os problemas de regressão e voltamos aos problemas de classificação. Mas desta vez, estamos interessando em problemas de classificação não-binária. Isto é, temos mais duas classes.
Nesse e no próximo problema, queremos prever o composto, primeiro a partir das expressões gênicas, e depois a partir dos MOAs ativados. 

Vamos nos limitar, porém, apenas aos nove compostos mais frequentes, para termos dados mais consistentes. Vamos descartar também o composto do grupo de controle.

In [70]:
#pegando a lista dos compostos mais frequentes
top_composto = df['composto'].value_counts()[1:9].index
top_composto

Index(['87d714366', '9f80f3f77', '8b87a7a83', '5628cb3ee', 'd08af5d4b',
       '292ab2c28', 'd50f18348', 'd1b47f29d'],
      dtype='object')

Filtrando nosso dataframe pelos compostos mais frequentes:

In [71]:
df_top = df_total.query('composto in @top_composto')
df_top

,id,tratamento,tempo,dose,composto,g0,g1,g2,g3,g4,g5,g6,g7,g8,g9,g10,g11,g12,g13,g14,g15,g16,g17,g18,g19,g20,g21,g22,g23,g24,g25,g26,g27,g28,g29,g30,g31,g32,g33,g34,...,c66,c67,c68,c69,c70,c71,c72,c73,c74,c75,c76,c77,c78,c79,c80,c81,c82,c83,c84,c85,c86,c87,c88,c89,c90,c91,c92,c93,c94,c95,c96,c97,c98,c99,n_moa,ativo_moa,Eh controle,Tempo eh 24,Tempo eh 48,Tempo eh 72
7,id_0020d0484,com_droga,48,D1,8b87a7a83,0.2711,0.5133,-0.1327,2.5950,0.6980,0.5846,-0.2633,-2.1490,0.4881,1.4750,-0.0488,-0.0561,0.1641,0.1749,-0.3464,-0.1811,0.5685,1.2970,1.0510,-0.9585,0.7911,-0.2991,2.7420,-0.5979,-1.0240,0.7268,0.1590,-1.9730,1.1340,-2.5720,0.2068,0.9992,-0.1958,0.2262,1.0140,...,0.8142,1.0990,1.0860,0.4011,-0.1342,0.4653,-0.1104,0.3935,1.2370,-0.5072,0.0351,0.5084,-0.1248,0.4351,0.1389,0.6726,0.6070,0.3431,-0.4116,0.1194,0.3420,0.1331,0.8541,0.6282,0.3230,-0.4140,0.2532,0.0513,0.860,1.4250,0.6633,0.4562,-0.9622,0.0260,1,1,0,0,1,0
16,id_002fb9c19,com_droga,48,D1,87d714366,8.7380,0.1914,2.4380,-0.2930,3.2010,1.4910,5.3920,-1.0420,-8.4860,-2.5790,-6.7470,3.6150,-1.4510,0.6909,-0.7623,-2.9400,-9.2850,-6.7110,-0.4195,1.1600,-1.3180,1.3660,-1.4020,-2.6220,0.7204,-2.5930,-4.0630,-0.8282,-3.5000,0.9206,-5.1450,9.8040,-2.6840,-2.4390,-0.1427,...,-10.0000,-10.0000,-10.0000,-8.8870,-10.0000,-10.0000,-10.0000,-10.0000,-4.9020,-9.9840,-5.5520,-9.9840,-9.9840,-9.9840,-9.9840,-9.9840,-9.9840,-9.9840,-9.9840,-9.9840,-9.7960,-8.1770,-9.8990,-9.9840,-9.9840,-9.9840,-9.9840,-9.9840,-9.984,-9.9840,-9.9840,-9.9840,-9.9840,-6.7840,2,1,0,0,1,0
39,id_0079d45d3,com_droga,24,D1,292ab2c28,1.6310,-2.1430,-0.0943,-1.1990,0.4869,-0.0935,1.2090,-3.8910,-1.5450,-2.2870,-2.2040,-0.8888,-1.1730,0.0444,-3.9430,0.1673,-0.9502,-1.1750,0.1686,3.8480,-2.9760,-0.0389,1.6330,-0.0655,1.3810,-2.4340,-1.8040,0.3945,-1.2080,-0.1963,-2.0170,0.6004,-2.5860,-1.4550,-0.4141,...,-4.7560,-0.8781,-3.7800,-1.6340,-3.9660,-1.7890,-3.2480,-2.6720,-1.3250,-2.3430,-3.2220,-2.5160,-2.3410,-2.4620,-3.8760,-2.2450,-3.9450,-1.7400,-4.4320,-3.6740,-2.9810,-4.5770,-1.8620,-1.9120,-2.9240,-5.7720,-1.9360,-2.8860,-2.768,-2.4840,-2.7020,-2.2530,-4.8360,-0.6461,0,0,0,1,0,0
44,id_00872d938,com_droga,48,D2,292ab2c28,0.3161,0.0464,0.1056,-2.1220,-0.5453,0.8377,-0.7082,-1.3680,1.0320,-0.7943,-1.3640,-1.8660,-1.5560,-0.7473,-3.8200,-0.0684,-0.0735,-0.7037,1.3380,1.2770,-3.6090,0.4893,1.0710,0.2883,-1.6720,-1.7010,-2.2230,-2.6830,-0.0307,0.5469,-2.5710,-0.6601,-1.3620,0.7463,-2.0630,...,-7.2140,-3.8900,-6.3190,-3.9200,-5.6400,-3.9420,-2.8180,-5.6200,-2.3530,-2.9780,-5.1180,-2.4600,-4.2850,-3.7600,-7.7470,-5.2730,-9.2110,-4.2840,-7.1290,-4.2520,-3.4490,-3.4050,-2.8660,-2.3470,-4.6080,-8.9130,-4.0500,-3.9080,-4.597,-4.0370,-3.9120,-3.6480,-3.5010,-5.4140,0,0,0,0,1,0
48,id_008b48693,com_droga,24,D1,5628cb3ee,2.5200,-2.0200,1.1710,2.0420,-1.8070,0.5109,-0.4123,0.3247,-1.6950,0.5503,0.1106,0.3894,0.2549,-1.1100,0.5913,0.7150,0.1877,-0.2982,-0.3119,1.3900,-1.0770,-2.3910,0.2645,1.1510,-0.3144,-0.2650,1.3330,-0.9137,-1.0100,0.4513,-0.1749,0.1450,0.2848,-0.1057,-0.1786,...,0.1877,0.4396,1.3060,0.4115,-0.1007,-1.1510,-0.6386,-0.4182,0.5802,1.0530,-0.1296,0.2485,-0.2013,-0.2202,0.4547,-0.2547,-0.6655,-0.3019,-0.1499,1.5890,-0.5297,-0.2534,-0.5758,-0.1000,-0.6200,0.0071,-0.3901,0.5272,-1.065,0.0289,-0.2725,-0.1511,-0.4635,-0.1365,1,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23782,id_ffb12855e,com_droga,72,D2,8b87a7a83,-1.1920,-0.4798,-0.2600,3.0090,-0.7221,0.9650,-0.4290,-0.8782,1.0180,-0.0630,0.6427,-0.0527,0.0457,0.0798,0.9841,-1.0940,1.8550,1.1920,4.0000,-0.0914,1.2400,-1.0150,2.3150,-0.4853,-0.7675,0.9849,0.4326,-0.4093,2.6750,-1.4490,1.1220,3.5690,-1.4320,0.9039,0.8561,...,0.5830,0.3561,1.2860,-1.2080,0.2401,0.2005,-0.9453,0.3551

Criando as variáveis:

In [72]:
y = df_top['composto']
x = df_top.drop(['composto', 'id', 'tempo', 'dose', 'tratamento'], axis = 1).iloc[:,0:771]

In [73]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x,y, test_size = 0.3)

In [74]:
lista_modelo = []
lista_score = []

### Regressão Logísitica

In [75]:
modelo = LogisticRegression()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  98.74411302982732 %


### DecisionTreeClassifier

In [76]:
modelo = DecisionTreeClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  87.91208791208791 %


###RandomFlorestClassifier

In [77]:
modelo = RandomForestClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  97.17425431711146 %


### XBG Classifier

In [78]:
modelo = XGBClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  97.48822605965462 %


### LGBM Classifier

In [79]:
modelo = LGBMClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  97.01726844583987 %


### GradientBoostClassifier


In [80]:
modelo = GradientBoostingClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  97.48822605965462 %


### KNeighbors Classifier

In [81]:
modelo = KNeighborsClassifier(n_neighbors=8)
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  91.67974882260597 %


### MLP Classifier

In [82]:
modelo = MLPClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  98.43014128728414 %


### Resumo

In [83]:
Tabela_Performace_Modelos = pd.DataFrame()
Tabela_Performace_Modelos['Modelo'] = lista_modelo
Tabela_Performace_Modelos['Score'] = lista_score

Tabela_Comparação_Modelos_Problema8 = Tabela_Performace_Modelos
Tabela_Comparação_Modelos_Problema8  

,Modelo,Score
0,LogisticRegression,0.987441
1,DecisionTreeClassifier,0.879121
2,RandomForestClassifier,0.971743
3,XGBClassifier,0.974882
4,LGBMClassifier,0.970173
5,GradientBoostingClassifier,0.974882
6,KNeighborsClassifier,0.916797
7,MLPClassifier,0.984301


Vemos então que todos os modelos tiveram desempenho excelente nesse problema. Importante notar que novamente, o modelo de árvore de decisão foi o menos preciso. 

## Problema 9: prevendo o composto a partir dos MOAs ativados

Para finalizar, em nosso último problema vamos construir modelos para prever o composto sabendo os MOAs que ele ativou.

In [84]:
#pegando a lista dos compostos mais frequentes
top_composto = df['composto'].value_counts()[1:9].index
top_composto

Index(['87d714366', '9f80f3f77', '8b87a7a83', '5628cb3ee', 'd08af5d4b',
       '292ab2c28', 'd50f18348', 'd1b47f29d'],
      dtype='object')

In [85]:
df_moas = pd.merge(df.drop(df.columns[5:], axis = 1), df_resultados, on='id')
df_moas

,id,tratamento,tempo,dose,composto,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,...,radiopaque_medium,raf_inhibitor,ras_gtpase_inhibitor,retinoid_receptor_agonist,retinoid_receptor_antagonist,rho_associated_kinase_inhibitor,ribonucleoside_reductase_inhibitor,rna_polymerase_inhibitor,serotonin_receptor_agonist,serotonin_receptor_antagonist,serotonin_reuptake_inhibitor,sigma_receptor_agonist,sigma_receptor_antagonist,smoothened_receptor_antagonist,sodium_channel_inhibitor,sphingosine_receptor_agonist,src_inhibitor,steroid,syk_inhibitor,tachykinin_antagonist,tgf-beta_receptor_inhibitor,thrombin_inhibitor,thymidylate_synthase_inhibitor,tlr_agonist,tlr_antagonist,tnf_inhibitor,topoisomerase_inhibitor,transient_receptor_potential_channel_antagonist,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,n_moa,ativo_moa
0,id_000644bb2,com_droga,24,D1,b68db1d53,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
1,id_000779bfc,com_droga,72,D1,df89a8e5a,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,com_droga,48,D1,18bb41b2c,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1
3,id_0015fd391,com_droga,48,D1,8c7f86626,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,com_droga,72,D2,7cbed3131,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,com_droga,24,D2,df1d0a5a1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
23810,id_fffb70c0c,com_droga,24,D2,ecf3b6b74,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
23811,id_fffc1c3f4,com_controle,48,D2,cacb2b860,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
23812,id_fffcb9e7c,com_droga,24,D1,8b87a7a83,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1


In [86]:
df_top = df_moas.query('composto in @top_composto')
df_top

,id,tratamento,tempo,dose,composto,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,...,radiopaque_medium,raf_inhibitor,ras_gtpase_inhibitor,retinoid_receptor_agonist,retinoid_receptor_antagonist,rho_associated_kinase_inhibitor,ribonucleoside_reductase_inhibitor,rna_polymerase_inhibitor,serotonin_receptor_agonist,serotonin_receptor_antagonist,serotonin_reuptake_inhibitor,sigma_receptor_agonist,sigma_receptor_antagonist,smoothened_receptor_antagonist,sodium_channel_inhibitor,sphingosine_receptor_agonist,src_inhibitor,steroid,syk_inhibitor,tachykinin_antagonist,tgf-beta_receptor_inhibitor,thrombin_inhibitor,thymidylate_synthase_inhibitor,tlr_agonist,tlr_antagonist,tnf_inhibitor,topoisomerase_inhibitor,transient_receptor_potential_channel_antagonist,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,n_moa,ativo_moa
7,id_0020d0484,com_droga,48,D1,8b87a7a83,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
16,id_002fb9c19,com_droga,48,D1,87d714366,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1
39,id_0079d45d3,com_droga,24,D1,292ab2c28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
44,id_00872d938,com_droga,48,D2,292ab2c28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
48,id_008b48693,com_droga,24,D1,5628cb3ee,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23782,id_ffb12855e,com_droga,72,D2,8b87a7a83,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
23791,id_ffd1fd119,com_droga,72,D2,87d714366,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1
23802,id_fff3976bd,com_droga,24,D1,87d714366,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1
23805,id_fff6df1c5,com_droga,48,D2,5628cb3ee,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1


In [87]:
y = df_top['composto']
x = df_top.drop(['composto', 'id', 'tempo', 'dose', 'tratamento'], axis = 1)

In [88]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x,y, test_size = 0.3)

In [89]:
lista_modelo = []
lista_score = []

### Regressão Logísitica

In [90]:
modelo = LogisticRegression()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  100.0 %


### DecisionTreeClassifier

In [92]:
modelo = DecisionTreeClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  100.0 %


###RandomFlorestClassifier

In [93]:
modelo = RandomForestClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  100.0 %


In [94]:
lista = (modelo.predict(x_teste) == y_teste)*1
lista.sum(), y_teste.shape

(637, (637,))

### XBG Classifier

In [95]:
modelo = XGBClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  100.0 %


### LGBM Classifier

In [96]:
modelo = LGBMClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  100.0 %


### GradientBoostClassfier


In [97]:
modelo = GradientBoostingClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  100.0 %


### KNeighbors Classifier

In [98]:
modelo = KNeighborsClassifier(n_neighbors=8)
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  100.0 %


### MLP Classifier

In [99]:
modelo = MLPClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  100.0 %


### Resumo

In [100]:
Tabela_Performace_Modelos = pd.DataFrame()
Tabela_Performace_Modelos['Modelo'] = lista_modelo
Tabela_Performace_Modelos['Score'] = lista_score

Tabela_Comparação_Modelos_Problema9 = Tabela_Performace_Modelos
Tabela_Comparação_Modelos_Problema9  

,Modelo,Score
0,LogisticRegression,1.0
1,DecisionTreeClassifier,1.0
2,RandomForestClassifier,1.0
3,XGBClassifier,1.0
4,LGBMClassifier,1.0
5,GradientBoostingClassifier,1.0
6,KNeighborsClassifier,1.0
7,MLPClassifier,1.0


Vemos então que nesse último problema os modelos tiveram uma performace perfeita, com 100% de acerto. Faz sentido, pois é de se esperar que compostos distintos ativem conjuntos distintos de MOAs.

